# This script is to download annotations for 1760 genes in E. coli K12

In [114]:
import re
import requests
from requests.adapters import HTTPAdapter, Retry
import os
from collections import defaultdict

import pandas as pd
import numpy as np

%config Completer.use_jedi = False # faster tab autofill

# Load gene list

In [3]:
pathdata = "/Users/zijianleowang/Downloads/Cornell/1Ongoing.Project/1000_genes_10_replicates_1_concentrations/data/Ecoli Lib"

In [6]:
lib = pd.read_excel(os.path.join(pathdata,"e.coli-promoter-library-0222.xls"),
                    sheet_name="ALL")
lib = lib.loc[~lib["Gene_Name"].isna(),:].iloc[:,:4]
lib.columns = ["Plate","Well","GeneName","Description"]

# remove duplicated GeneName
lib = lib.drop_duplicates("GeneName")

# remove row for any NaN in Plate or Well or GeneName
lib = lib.dropna()
lib = lib.reset_index()
genes = lib["GeneName"].tolist()

# preview lib
lib

,index,Plate,Well,GeneName,Description
0,0,109-AZ01,D4,aer,"aerotaxis sensor receptor, flavoprotein"
1,1,109-AZ01,D1,amiC,N-acetylmuramoyl-L-alanine amidase (2nd module)
2,2,109-AZ01,B7,argA,N-alpha-acetylglutamate synthase (amino-acid a...
3,3,109-AZ01,C6,argC,"N-acetyl-gamma-glutamylphosphate reductase, NA..."
4,4,109-AZ01,F5,argD,acetylornithine transaminase (NAcOATase and Da...
...,...,...,...,...,...
1795,1923,109-AZ21,A9,lacA,"galactoside O-acetyltransferase monomer , subu..."
1796,1924,109-AZ21,A3,yafZ,hypothetical protein
1797,1925,109-AZ21,A6,ybjG,putative permease
1798,1927,109-AZ21,A2,yfbM,conserved protein


# Download annotations of the E. coli gene list by [Uniprot Python API](https://www.uniprot.org/help/api_queries)

In [50]:
# code from official API tutorial: https://www.uniprot.org/help/api_queries
"""
View the results in the browser here: https://www.uniprot.org/uniprotkb?query=Insulin%20AND%20(reviewed:true)
Click Download in the toolbar
For this example we want the following selected:
Download all
Format: TSV
Compressed: No
Click Generate URL for API
Under the header API URL using the search endpoint click Copy to get the URL which will be used in the following snippet.
Always use size=500 as this will provide fast performance.
"""
def download_annotation(pathdata,batchsize,pathout,dbfile,logp,genes):
    """
    download the uniprot annotation for the ecoli database
    
    
    """
    re_next_link = re.compile(r'<(.+)>; rel="next"')
    retries = Retry(total=5, backoff_factor=0.25, status_forcelist=[500, 502, 503, 504])
    session = requests.Session()
    session.mount("https://", HTTPAdapter(max_retries=retries))

    def get_next_link(headers):
        if "Link" in headers:
            match = re_next_link.match(headers["Link"])
            if match:
                return match.group(1)

    def get_batch(batch_url):
        while batch_url:
            response = session.get(batch_url)
            response.raise_for_status()
            total = response.headers["x-total-results"]
            yield response, total
            batch_url = get_next_link(response.headers)

    # download top1 query of a certain gene in E.coli K12
    logfile = open(logp,"w")
    progress = 0

    with open(dbfile, 'w') as f:
        for i,gene in enumerate(genes):
            # below is for (gene:gene) AND (organism_id:83333); the orgid 83333 is for Escherichia coli K12
            url = 'https://rest.uniprot.org/uniprotkb/search?fields=accession%2Creviewed%2Cid%2Cprotein_name%2Cgene_names%2'+\
            'Corganism_name%2Clength%2Cgo_p%2Cgo%2Cgo_id%2Cgo_f%2Cgo_c%2Ccc_interaction%2Ccc_subunit%2Cft_intramem%2' +\
            'Ccc_subcellular_location%2Cft_transmem%2Cft_topo_dom%2Cft_chain%2Cft_crosslnk%2Cft_disulfid%2Cft_carbohyd%2'+\
            'Cft_peptide%2Cft_mod_res%2Cft_lipid%2Cft_init_met%2Ccc_ptm%2Cft_propep%2Cft_signal%2Cft_transit&format=tsv' +\
            '&query=%28%28gene%3A{}%29%20AND%20%28organism_id%3A83333%29%29&size={}'.format(gene,batchsize)

            for batch, total in get_batch(url): 

                lines = batch.text.splitlines() # if len(lines) == 1, that means nothing found but only field name has been retrieved

                # add field name
                if not progress:  
                    print("OriginalGeneName\t"+lines[0], # add one more field for original gene name
                          file=f)

                # notify if no gene found
                if len(lines)==1:
                    print("%s gene with index of %i in original e.coli-promoter-library-0222.xls db is not found in UniProt"%(gene,i),
                          file=logfile)

                # save record if it found a gene in UniProt
                if len(lines) >=2:
                    print("%s\t"%gene+lines[1], # add original gene name
                          file=f) # only print ranked one record

                progress += len(lines[1:])
                #print(f'{progress} / {total}')
                break # only download one batch

    logfile.close()

In [51]:
# hyperparameter - step 1; originally
batchsize = 1 
pathout = os.path.join(pathdata,"Uniprot_Annotation") # directory for downloaded database
dbfile = os.path.join(pathout,'Ecoli.tsv')
logp = os.path.join(pathout,'log.uniprotdownload.txt')
download_annotation(pathdata,batchsize,pathout,dbfile,logp,genes)

In [52]:
db = pd.read_csv(dbfile,sep='\t')
db

,OriginalGeneName,Entry,Reviewed,Entry Name,Protein names,Gene Names,Organism,Length,Gene Ontology (biological process),Gene Ontology (GO),...,Disulfide bond,Glycosylation,Peptide,Modified residue,Lipidation,Initiator methionine,Post-translational modification,Propeptide,Signal peptide,Transit peptide
0,aer,P50466,reviewed,AER_ECOLI,Aerotaxis receptor,aer air yqjJ b3072 JW3043,Escherichia coli (strain K12),506,chemotaxis [GO:0006935]; positive aerotaxis [G...,plasma membrane [GO:0005886]; identical protei...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,amiC,P63883,reviewed,AMIC_ECOLI,N-acetylmuramoyl-L-alanine amidase AmiC (EC 3....,amiC ygdN b2817 JW5449,Escherichia coli (strain K12),417,cell wall organization [GO:0071555]; FtsZ-depe...,outer membrane-bounded periplasmic space [GO:0...,...,NaN,NaN,NaN,NaN,NaN,NaN,PTM: Exported by the Tat system. The position ...,NaN,"SIGNAL 1..31; /note=""Tat-type signal""; /eviden...",NaN
2,argA,P0A6C5,reviewed,ARGA_ECOLI,Amino-acid acetyltransferase (EC 2.3.1.1) (N-a...,argA b2818 JW2786,Escherichia coli (strain K12),443,arginine biosynthetic process [GO:0006526],cytoplasm [GO:0005737]; acetyl-CoA:L-glutamate...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,argC,P11446,reviewed,ARGC_ECOLI,N-acetyl-gamma-glutamyl-phosphate reductase (A...,argC b3958 JW3930,Escherichia coli (strain K12),334,arginine biosynthetic process [GO:0006526],cytoplasm [GO:0005737]; N-acetyl-gamma-glutamy...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,argD,P18335,reviewed,ARGD_ECOLI,Acetylornithine/succinyldiaminopimelate aminot...,argD dapC dtu b3359 JW3322,Escherichia coli (strain K12),406,arginine biosynthetic process [GO:0006526]; ly...,cytoplasm [GO:0005737]; identical protein bind...,...,NaN,NaN,NaN,"MOD_RES 255; /note=""N6-(pyridoxal phosphate)ly...",NaN,"INIT_MET 1; /note=""Removed""; /evidence=""ECO:00...",NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1717,lacA,P07464,reviewed,THGA_ECOLI,Galactoside O-acetyltransferase (GAT) (EC 2.3....,lacA b0342 JW0333,Escherichia coli (strain K12),203,lactose biosynthetic process [GO:0005989],cytoplasm [GO:0005737]; galactoside O-acetyltr...,...,NaN,NaN,NaN,NaN,NaN,"INIT_MET 1; /note=""Removed; Partial""; /evidenc...",PTM: The N-terminus of this protein is heterog...,NaN,NaN,NaN
1718,yafZ,P77206,reviewed,YAFZ_ECOLI,UPF0380 protein YafZ,yafZ b0252 JW0242,Escherichia coli (strain K12),273,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1719,ybjG,P75806,reviewed,YBJG_ECOLI,Putative undecaprenyl-diphosphatase YbjG (EC 3...,ybjG b0841 JW5112,Escherichia coli (strain K12),198,cell wall organization [GO:0071555]; peptidogl...,plasma membrane [GO:0005886]; undecaprenyl-dip...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1720,yfbM,P76483,reviewed,YFBM_ECOLI,Protein YfbM,yfbM b2272 JW2267,Escherichia coli (strain K12),167,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Curated the genelist manually by checking the logfile
Purpose: To fetch nodal information from UniProt db for each information to compute BNVAR model but also the 1760 gene expression file need to be aligned with original name

1. The gene name is updated
- *alsI* is not found but by checking the description, it is for ribose 5-phosphate isomerase B, also acts on allose (allose 6-phosphate isomerase). So, I searched online and found its name as rpiB in E. coli K12. https://www.uniprot.org/uniprotkb/P37351/entry

2. The gene relevant protein is not recorded in UniProt, so just keep it empty with no information

3. The gene is deleted because it is a phantom gene

Then, I create 2 new excel file
1. PATH/Uniprot_Annotation/e.coli-library-curation-06052023.xlsx 
- Tab "Missing-Curated" for case1
- Tab "Missing" for case2
- Tab "Deleted" for case3


2. create a newly curated database in PATH/Uniprot_Annotation/e.coli-promoter-library-curated-06052023.xls
This novel db would have fields name of Plate, Well, GeneName, Description, UniProtGeneName

Later it will be combined with downloaded record information

Download the record for case1


## download case1 missing-curated record from uniprot

In [53]:
genes_curated = pd.read_excel(os.path.join(pathout,"e.coli-library-curation-06052023.xlsx"),sheet_name="Missing-Curated")
genes_curated = genes_curated["RevisedGeneName"].tolist()
genes_curated

['rpiB',
 'pykF',
 'tdcG',
 'gatR',
 'insB4',
 'wecF',
 'yfjV',
 'gss',
 'insA7',
 'yehF',
 'insA3',
 'insA1',
 'insA2',
 'groES']

In [54]:
# hyperparameter - step 2; the curated missing record
dbfilecurated = os.path.join(pathout,'Ecoli_curated.tsv')
logpcurated = os.path.join(pathout,'log.uniprotdownload.curated.txt')
download_annotation(pathdata,batchsize,pathout,
                    dbfilecurated,logpcurated,genes_curated)

In [55]:
dbcurated = pd.read_csv(dbfilecurated,sep='\t')
dbcurated

,OriginalGeneName,Entry,Reviewed,Entry Name,Protein names,Gene Names,Organism,Length,Gene Ontology (biological process),Gene Ontology (GO),...,Disulfide bond,Glycosylation,Peptide,Modified residue,Lipidation,Initiator methionine,Post-translational modification,Propeptide,Signal peptide,Transit peptide
0,rpiB,P37351,reviewed,RPIB_ECOLI,Ribose-5-phosphate isomerase B (EC 5.3.1.6) (P...,rpiB yjcA b4090 JW4051,Escherichia coli (strain K12),149,D-allose catabolic process [GO:0019316]; pento...,allose 6-phosphate isomerase activity [GO:0008...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,pykF,P0AD61,reviewed,KPYK1_ECOLI,Pyruvate kinase I (EC 2.7.1.40) (PK-1),pykF b1676 JW1666,Escherichia coli (strain K12),470,glycolytic process [GO:0006096]; phosphorylati...,cytoplasm [GO:0005737]; cytosol [GO:0005829]; ...,...,NaN,NaN,NaN,"MOD_RES 76; /note=""N6-acetyllysine""; /evidence...",NaN,NaN,NaN,NaN,NaN,NaN
2,tdcG,P42630,reviewed,TDCG_ECOLI,L-serine dehydratase TdcG (SDH) (EC 4.3.1.17) ...,tdcG yhaP yhaQ b4471 JW5520,Escherichia coli (strain K12),454,amino acid catabolic process [GO:0009063]; glu...,"4 iron, 4 sulfur cluster binding [GO:0051539];...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,gatR,P36930,reviewed,GATR_ECOLI,Putative galactitol utilization operon repressor,gatR b4498 JW5340/JW2074 b2087/b2090,Escherichia coli (strain K12),112,NaN,DNA binding [GO:0003677]; DNA-binding transcri...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,insB4,P57998,reviewed,INSB4_ECOLI,Insertion element IS1 4 protein InsB (IS1d),insB4 b0988 JW0972,Escherichia coli (strain K12),167,"transposition, DNA-mediated [GO:0006313]",DNA binding [GO:0003677]; transposase activity...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,wecF,P56258,reviewed,WECF_ECOLI,TDP-N-acetylfucosamine:lipid II N-acetylfucosa...,wecF rffT yifM b4481 JW5596,Escherichia coli (strain K12),359,enterobacterial common antigen biosynthetic pr...,"plasma membrane [GO:0005886]; 4-acetamido-4,6-...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,yfjV,A0A0S1EZS3,unreviewed,A0A0S1EZS3_ECOLI,Arsenical pump membrane protein,yfjV,Escherichia coli (strain K12),66,NaN,plasma membrane [GO:0005886]; arsenite transme...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,gss,P0AES0,reviewed,GSP_ECOLI,Bifunctional glutathionylspermidine synthetase...,gss gsp b2988 JW2956,Escherichia coli (strain K12),619,glutathione metabolic process [GO:0006749]; sp...,cytosol [GO:0005829]; ATP binding [GO:0005524]...,...,NaN,NaN,NaN,"MOD_RES 59; /note=""Cysteine sulfenic acid (-SO...",NaN,"INIT_MET 1; /note=""Removed""; /evidence=""ECO:00...",PTM: Oxidation of Cys-59 to sulfenic acid duri...,NaN,NaN,NaN
8,insA7,P19767,reviewed,INSA7_ECOLI,Insertion element IS1 7 protein InsA (IS1f),insA7 b4294 JW4254,Escherichia coli (strain K12),91,"transposition, DNA-mediated [GO:0006313]","transposition, DNA-mediated [GO:0006313]",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,yehF,P33345,reviewed,YEHF_ECOLI,Protein YehF,yehF dinO molR sosF b2115 b2116/b2117 b4499,Escherichia coli (strain K12),274,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## combine the original and curated db

In [64]:
db_uniprot_all = pd.concat([dbcurated,db],axis=0)
db_uniprot_all.reset_index(inplace=True,drop=True)
db_uniprot_all.drop_duplicates(inplace=True,ignore_index=True)
db_uniprot_all.to_excel(os.path.join(pathout,"Ecoli.uniprot.all.xlsx"),
    header=True,index=False)
db_uniprot_all

,OriginalGeneName,Entry,Reviewed,Entry Name,Protein names,Gene Names,Organism,Length,Gene Ontology (biological process),Gene Ontology (GO),...,Disulfide bond,Glycosylation,Peptide,Modified residue,Lipidation,Initiator methionine,Post-translational modification,Propeptide,Signal peptide,Transit peptide
0,rpiB,P37351,reviewed,RPIB_ECOLI,Ribose-5-phosphate isomerase B (EC 5.3.1.6) (P...,rpiB yjcA b4090 JW4051,Escherichia coli (strain K12),149,D-allose catabolic process [GO:0019316]; pento...,allose 6-phosphate isomerase activity [GO:0008...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,pykF,P0AD61,reviewed,KPYK1_ECOLI,Pyruvate kinase I (EC 2.7.1.40) (PK-1),pykF b1676 JW1666,Escherichia coli (strain K12),470,glycolytic process [GO:0006096]; phosphorylati...,cytoplasm [GO:0005737]; cytosol [GO:0005829]; ...,...,NaN,NaN,NaN,"MOD_RES 76; /note=""N6-acetyllysine""; /evidence...",NaN,NaN,NaN,NaN,NaN,NaN
2,tdcG,P42630,reviewed,TDCG_ECOLI,L-serine dehydratase TdcG (SDH) (EC 4.3.1.17) ...,tdcG yhaP yhaQ b4471 JW5520,Escherichia coli (strain K12),454,amino acid catabolic process [GO:0009063]; glu...,"4 iron, 4 sulfur cluster binding [GO:0051539];...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,gatR,P36930,reviewed,GATR_ECOLI,Putative galactitol utilization operon repressor,gatR b4498 JW5340/JW2074 b2087/b2090,Escherichia coli (strain K12),112,NaN,DNA binding [GO:0003677]; DNA-binding transcri...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,insB4,P57998,reviewed,INSB4_ECOLI,Insertion element IS1 4 protein InsB (IS1d),insB4 b0988 JW0972,Escherichia coli (strain K12),167,"transposition, DNA-mediated [GO:0006313]",DNA binding [GO:0003677]; transposase activity...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1731,lacA,P07464,reviewed,THGA_ECOLI,Galactoside O-acetyltransferase (GAT) (EC 2.3....,lacA b0342 JW0333,Escherichia coli (strain K12),203,lactose biosynthetic process [GO:0005989],cytoplasm [GO:0005737]; galactoside O-acetyltr...,...,NaN,NaN,NaN,NaN,NaN,"INIT_MET 1; /note=""Removed; Partial""; /evidenc...",PTM: The N-terminus of this protein is heterog...,NaN,NaN,NaN
1732,yafZ,P77206,reviewed,YAFZ_ECOLI,UPF0380 protein YafZ,yafZ b0252 JW0242,Escherichia coli (strain K12),273,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1733,ybjG,P75806,reviewed,YBJG_ECOLI,Putative undecaprenyl-diphosphatase YbjG (EC 3...,ybjG b0841 JW5112,Escherichia coli (strain K12),198,cell wall organization [GO:0071555]; peptidogl...,plasma membrane [GO:0005886]; undecaprenyl-dip...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1734,yfbM,P76483,reviewed,YFBM_ECOLI,Protein YfbM,yfbM b2272 JW2267,Escherichia coli (strain K12),167,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Combine UniProt DB with original DB

In [60]:
db_original_curated = pd.read_excel(os.path.join(pathout,"e.coli-promoter-library-curated-06052023.xls"),
                                   sheet_name="ALL")
db_original_curated

,Plate,Well,OriginalGeneName,Description,UniProtGeneName
0,109-AZ01,D3,b3007,unknown CDS,b3007
1,109-AZ01,A12,Empty,Empty well,Empty
2,109-AZ01,C10,U139,promoterless strain,U139
3,109-AZ01,F3,U66,promoterless strain,U66
4,109-AZ02,E11,pagB,unknown CDS,pagB
...,...,...,...,...,...
1919,109-AZ21,A3,yafZ,hypothetical protein,yafZ
1920,109-AZ21,A6,ybjG,putative permease,ybjG
1921,109-AZ21,A8,yebF,unknown CDS,yebF
1922,109-AZ21,A2,yfbM,conserved protein,yfbM


In [93]:
db_all  = db_original_curated.merge(db_uniprot_all,how="inner")
db_all.reset_index(inplace=True,drop=True)
db_all.drop_duplicates(inplace=True,ignore_index=True)
db_all.to_excel(os.path.join(pathout,"Ecoli.all.xlsx"),
    header=True,index=False)
db_all

,Plate,Well,OriginalGeneName,Description,UniProtGeneName,Entry,Reviewed,Entry Name,Protein names,Gene Names,...,Disulfide bond,Glycosylation,Peptide,Modified residue,Lipidation,Initiator methionine,Post-translational modification,Propeptide,Signal peptide,Transit peptide
0,109-AZ01,D4,aer,"aerotaxis sensor receptor, flavoprotein",aer,P50466,reviewed,AER_ECOLI,Aerotaxis receptor,aer air yqjJ b3072 JW3043,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,109-AZ01,D1,amiC,N-acetylmuramoyl-L-alanine amidase (2nd module),amiC,P63883,reviewed,AMIC_ECOLI,N-acetylmuramoyl-L-alanine amidase AmiC (EC 3....,amiC ygdN b2817 JW5449,...,NaN,NaN,NaN,NaN,NaN,NaN,PTM: Exported by the Tat system. The position ...,NaN,"SIGNAL 1..31; /note=""Tat-type signal""; /eviden...",NaN
2,109-AZ01,B7,argA,N-alpha-acetylglutamate synthase (amino-acid a...,argA,P0A6C5,reviewed,ARGA_ECOLI,Amino-acid acetyltransferase (EC 2.3.1.1) (N-a...,argA b2818 JW2786,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,109-AZ01,C6,argC,"N-acetyl-gamma-glutamylphosphate reductase, NA...",argC,P11446,reviewed,ARGC_ECOLI,N-acetyl-gamma-glutamyl-phosphate reductase (A...,argC b3958 JW3930,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,109-AZ01,F5,argD,acetylornithine transaminase (NAcOATase and Da...,argD,P18335,reviewed,ARGD_ECOLI,Acetylornithine/succinyldiaminopimelate aminot...,argD dapC dtu b3359 JW3322,...,NaN,NaN,NaN,"MOD_RES 255; /note=""N6-(pyridoxal phosphate)ly...",NaN,"INIT_MET 1; /note=""Removed""; /evidence=""ECO:00...",NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1780,109-AZ21,A9,lacA,"galactoside O-acetyltransferase monomer , subu...",lacA,P07464,reviewed,THGA_ECOLI,Galactoside O-acetyltransferase (GAT) (EC 2.3....,lacA b0342 JW0333,...,NaN,NaN,NaN,NaN,NaN,"INIT_MET 1; /note=""Removed; Partial""; /evidenc...",PTM: The N-terminus of this protein is heterog...,NaN,NaN,NaN
1781,109-AZ21,A3,yafZ,hypothetical protein,yafZ,P77206,reviewed,YAFZ_ECOLI,UPF0380 protein YafZ,yafZ b0252 JW0242,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1782,109-AZ21,A6,ybjG,putative permease,ybjG,P75806,reviewed,YBJG_ECOLI,Putative undecaprenyl-diphosphatase YbjG (EC 3...,ybjG b0841 JW5112,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1783,109-AZ21,A2,yfbM,conserved protein,yfbM,P76483,reviewed,YFBM_ECOLI,Protein YfbM,yfbM b2272 JW2267,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# process the database

### Catogorize genes into GO biological process based on [Gene Ontology (biological process)](https://www.uniprot.org/help/gene_ontology)

In [67]:
# preview it
GObio = db['Gene Ontology (biological process)']
print(GObio)
# each gene is involved in multiple biological processes

0       chemotaxis [GO:0006935]; positive aerotaxis [G...
1       cell wall organization [GO:0071555]; FtsZ-depe...
2              arginine biosynthetic process [GO:0006526]
3              arginine biosynthetic process [GO:0006526]
4       arginine biosynthetic process [GO:0006526]; ly...
                              ...                        
1780            lactose biosynthetic process [GO:0005989]
1781                                                  NaN
1782    cell wall organization [GO:0071555]; peptidogl...
1783                                                  NaN
1784                     DNA damage response [GO:0006974]
Name: Gene Ontology (biological process), Length: 1785, dtype: object


In [97]:
# total unique GO in this Ecoli db
allgo = set()
goindex = [] # index for gene with GO id
log = open(os.path.join(pathout,"GO.annotate.log.txt",),"w") # the log information to record what gene has no GO annotation in Uniprot
for i,tempgene in enumerate(GObio.tolist()): # tempgo is the GOs for each gene

    if type(tempgene)==str:
        goindex.append(i)
        tempgolist = tempgene.split(";")
        for goi in tempgolist:
            allgo.add(goi)
            
    else:
        print("------------------",file=log)
        print("i == %i, no GO detected \n"%i,file=log)
        print(db_all.iloc[i,:6],file=log)

print("-------------",file=log)
print("%i genes has been found with GO annotation"%len(goindex),file=log)
print("In total of %i unique GO biological processes detected in this E. coli K12 database",file=log)
print("gene index for E.coli database with GO annotations is as below",file=log)
print(goindex,file=log)
log.close()
allgo = list(allgo)
with open(os.path.join(pathout,"all.GO.txt"),"w") as f:
    for goi in allgo:
        print(goi,file=f)

In [78]:
# preview the unique GO 
print(allgo[:10])
print("total of %i GO biological process"%len(allgo))

['cellular respiration [GO:0045333]', ' galactitol catabolic process [GO:0019404]', ' DNA replication [GO:0006260]', 'D-xylose metabolic process [GO:0042732]', ' DNA recombination [GO:0006310]', ' response to amino acid [GO:0043200]', ' valine biosynthetic process [GO:0009099]', ' cellular response to starvation [GO:0009267]', 'regulation of phosphorelay signal transduction system [GO:0070297]', ' siderophore biosynthetic process [GO:0019290]']
total of 1445 GO biological process


In [89]:
# categorize them based on shape of genes x # GO biological process

dfgo = pd.DataFrame(np.zeros([db_all.shape[0],
                              len(allgo)])
                   ).astype("int") # dataframe to save GO biological process (0-th col) and gene index (1-th col)
dfgo.columns = allgo
dfgo.index = db_all["OriginalGeneName"].tolist()

for tempi, tempgo in enumerate(GObio):
    if type(tempgo) == str:
        tempgolist = tempgo.split(";")
        if len(tempgolist)>=1:
            for tempgoi in tempgolist:
                dfgo[tempgoi].iloc[tempi] = 1
        

In [95]:
dfgo.to_excel(os.path.join(pathout,"Ecoli.all.GO.xlsx"),
    header=True,index=True)

# Filter arsen relevant GO
There is 1445 unique GOs in this library, which is too much to handle. Let's filter arsen and stress relevant

In [103]:
with open("/Users/zijianleowang/Downloads/Cornell/1Ongoing.Project/1000_genes_10_replicates_1_concentrations/data/Ecoli Lib/Uniprot_Annotation/all.GO.txt") as f:
    allgo = f.readlines()
    allgo = [i.replace("\n","") for i in allgo]

dfgo = pd.read_excel("/Users/zijianleowang/Downloads/Cornell/1Ongoing.Project/1000_genes_10_replicates_1_concentrations/data/Ecoli Lib/Uniprot_Annotation/Ecoli.all.GO.xlsx",
                      index_col=0)

In [132]:

filt_kw = ["arsen","stress"]# keyword list for filtration
filt_go = defaultdict(list,{ k:{} for k in filt_kw }) # initialize

for kw in filt_kw:
    goi = [i for i in allgo if kw in i ] # GO names under certain filtration kw criteria

    # only one GO under this filteration
    if len(goi) == 1:
        goj = goi[0]
        genei = dfgo[goj] # the bool series of gene for GO i
        genei = genei[genei==1].index.tolist() # the selected gene sets for i-th GO
        filt_go[kw][goj] = genei

    # >=2 GOs under this filteration
    elif len(goi) >= 2:
        for goj in goi:
            genei = dfgo[goj] # the bool series of gene for GO i
            genei = genei[genei==1].index.tolist() # the selected gene sets for i-th GO
            filt_go[kw][goj] = genei

In [136]:
filt_go["stress"].keys()

dict_keys(['response to osmotic stress [GO:0006970]', ' cellular response to cell envelope stress [GO:0036460]', ' cellular response to oxidative stress [GO:0034599]', ' response to salt stress [GO:0009651]', 'cellular response to cell envelope stress [GO:0036460]', ' cellular stress response to acidic pH [GO:1990451]', ' cellular response to glucose-phosphate stress [GO:0036448]', ' stress response to copper ion [GO:1990169]', 'negative regulation of translation in response to stress [GO:0032055]', 'cellular response to osmotic stress [GO:0071470]', ' cellular response to osmotic stress [GO:0071470]', 'cellular response to oxidative stress [GO:0034599]', ' response to oxidative stress [GO:0006979]', 'response to oxidative stress [GO:0006979]', ' cellular stress response to acid chemical [GO:0097533]', 'cellular stress response to acidic pH [GO:1990451]', ' response to osmotic stress [GO:0006970]', ' regulation of cellular response to stress [GO:0080135]', ' response to nitrosative str

In [110]:
db_all = pd.read_excel("/Users/zijianleowang/Downloads/Cornell/1Ongoing.Project/1000_genes_10_replicates_1_concentrations/data/Ecoli Lib/Uniprot_Annotation/Ecoli.all.GO.xlsx",
                      index_col=0)

,cellular respiration [GO:0045333],galactitol catabolic process [GO:0019404],DNA replication [GO:0006260],D-xylose metabolic process [GO:0042732],DNA recombination [GO:0006310],response to amino acid [GO:0043200],valine biosynthetic process [GO:0009099],cellular response to starvation [GO:0009267],regulation of phosphorelay signal transduction system [GO:0070297],siderophore biosynthetic process [GO:0019290],...,peptidoglycan-protein cross-linking [GO:0018104],positive aerotaxis [GO:0052131],response to zinc ion [GO:0010043],ribosomal large subunit assembly [GO:0000027],carbohydrate derivative transport [GO:1901264],co-translational protein modification [GO:0043686],RNA secondary structure unwinding [GO:0010501],lipoprotein metabolic process [GO:0042157],copper ion transmembrane transport [GO:0035434],protein autoprocessing [GO:0016540]
aer,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
amiC,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
argA,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
argC,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
argD,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
lacA,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
yafZ,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ybjG,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
yfbM,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [113]:
genei = dfgo['response to osmotic stress [GO:0006970]']
genei[genei==1]

osmB    1
otsB    1
osmE    1
Name: response to osmotic stress [GO:0006970], dtype: int64

In [ ]:
gshA